Import required libraries

In [48]:
# Agent libs
from langchain.agents import initialize_agent, Tool
from langchain.tools import StructuredTool
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel
# Environment libs
import os

from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TRANSLATOR_MODEL = os.getenv("TRANSLATOR_MODEL")
#assert OPENAI_API_KEY, "Please set the OPENAI_API_KEY environment variable"
#assert TRANSLATOR_MODEL, "Please set the TRANSLATOR_MODEL environment variable"

## Agents

- Generator:
  - Context: Seed sentences
  - Action: Generate a combination of sentences
- Translator:
  - Context: Example translations
  - Action: Translate source sentence to target language

In [55]:
generator = ChatOpenAI(model="gpt-4o-mini")

def generate_text(seed_sentences: list[str]) -> str:
    prompt = PromptTemplate.from_template("Sentence {i}: {sentence}")
    examples = [{"i": i, "sentence": sentence} for i, sentence in enumerate(seed_sentences)]
    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=prompt,
        prefix="Create a new different sentence that contains words from the following sentences. The new sentence should make logical sense and be grammatically correct sentence. The new sentence should be the same length as the original sentences.\n",
        suffix="\nNew sentence:"
    )
    prompt_text = few_shot_prompt.format()
    # DEBUG: print(prompt_text)
    response = generator.invoke(prompt_text)
    return response.content # type: ignore




translator = ChatOpenAI(model="gpt-4o-mini") # PLACEHOLDER

class TranslationInput(BaseModel):
    text: str
    examples: list[tuple[str, str]]

def translate_text(input_data : TranslationInput) -> str:
    translation_prompt = PromptTemplate.from_template("[EN]{source_text}[/EN]\n[ANG]{target_text}[/ANG]")
    translation_examples = [{"source_text": source_text, "target_text": target_text} for source_text, target_text in input_data.examples]

    few_shot_prompt = FewShotPromptTemplate(
        examples=translation_examples,
        example_prompt=translation_prompt,
        prefix="Translate the following text from English to Anglo-Saxon:\n",
        suffix=f"\n[EN]{input_data.text}[/EN]\n[ANG]",
        input_variables=["text"]
    )
    prompt_text = few_shot_prompt.format(text=input_data.text)
    # DEBUG: print(prompt_text)
    response = translator.invoke(prompt_text)
    return response.content # type: ignore

In [56]:
# Tools

generator_tool = Tool(
    name="generate_text",
    description="Generate a new sentence that contains words from the given sentences.",
    input_variables=["seed_sentences"],
    output_variables=["new_sentence"],
    func=generate_text
)

translator_tool = Tool(
    name="translate_text",
    description="Translate the given text from English to Anglo-Saxon.",
    input_variables=["data"],
    output_variables=["translation"],
    func=translate_text
)


In [57]:
# Initalize pipeline/agents
def agent_pipeline(example_sentences: list[tuple[str, str]]):

    seed_sentences = [sentence[0] for sentence in example_sentences]

    new_english_sentence = generator_tool.run({"seed_sentences":seed_sentences})

    data = TranslationInput(text=new_english_sentence, examples=example_sentences)

    new_anglo_saxon_sentence = translator_tool.run({"data":data})
    return new_english_sentence, new_anglo_saxon_sentence


In [58]:
# USAGE
examples=[
  ("now was one of the twelve of christ's companions, who was called judas, seduced by the instigation of the devil, and he went to the jewish people, and consulted with them how he might betray christ unto them.", "ða wearð an ðæra twelfa cristes geferena, se wæs iudas gehaten, ðurh deofles tihtinge beswicen, and he eode to ðam iudeiscum folce, and smeade wið hi, hu he crist him belæwan mihte."),
  ("in that battle fell the power and dignity of the medes.", "and on ðæm gefeohte meða cræft and heora duguð gefeoll."),
  ("46 shillings shall be given as compensation for a wound on the head, if both bones are pierced.", "xl. heafodwunde to bote, gif ða ban beoð butu ðyrel, xxx scillinga geselle him mon.")
]

new_english_sentence, new_anglo_saxon_sentence = agent_pipeline(examples)
print(new_english_sentence)
print(new_anglo_saxon_sentence)

Create a new different sentence that contains words from the following sentences. The new sentence should make logical sense and be grammatically correct sentence. The new sentence should be the same length as the original sentences.


Sentence 0: now was one of the twelve of christ's companions, who was called judas, seduced by the instigation of the devil, and he went to the jewish people, and consulted with them how he might betray christ unto them.

Sentence 1: in that battle fell the power and dignity of the medes.

Sentence 2: 46 shillings shall be given as compensation for a wound on the head, if both bones are pierced.


New sentence:
Translate the following text from English to Anglo-Saxon:


[EN]now was one of the twelve of christ's companions, who was called judas, seduced by the instigation of the devil, and he went to the jewish people, and consulted with them how he might betray christ unto them.[/EN]
[ANG]ða wearð an ðæra twelfa cristes geferena, se wæs iudas gehaten, ðu